<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/model_assisted_labeling/tiled_imagery_mal.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/model_assisted_labeling/tiled_imagery_mal.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Tiled Imagery Annotation Import
* This notebook will provide examples of each supported annotation type for tiled imagery assets, and also cover MAL and Label Import methods:

Supported annotations that can be uploaded through the SDK: 
  * Point 
  * Polygon
  * Bounding Box 
  * Classification radio 
  * Classification checklist 
  * Classification free-form text

**Not** supported:
  * Segmentation mask


MAL and Label Import: 

* Model-assisted labeling - used to provide pre-annotated data for your labelers. This will enable a reduction in the total amount of time to properly label your assets. Model-assisted labeling does not submit the labels automatically, and will need to be reviewed by a labeler for submission.
* Label Import - used to provide ground truth labels. These can in turn be used and compared against prediction labels, or used as benchmarks to see how your labelers are doing.

For information on what types of annotations are supported per data type, refer to this documentation:
    * https://docs.labelbox.com/docs/model-assisted-labeling#option-1-import-via-python-annotation-types-recommended

Notes:
 * This notebook uses the Slippy Maps format
 * If you are importing more than 1,000 annotations at a time, consider submitting separate jobs, as they can take longer than other annotation types to import.
 * Wait until the import job is complete before opening the Editor to make sure all annotations are imported properly.
 * You may need to refresh your browser in order to see the results of the import job.

### Setup

In [ ]:
!pip install -q 'labelbox[data]'

In [2]:
import os

import uuid
import numpy as np
from PIL import Image
import cv2
import ndjson
import gdal


from labelbox.data.serialization.ndjson.converter import NDJsonConverter
from labelbox.schema.queue_mode import QueueMode
from labelbox.schema.media_type import MediaType
from labelbox.data.annotation_types import (
    Label, ImageData, ObjectAnnotation, 
    Rectangle, Point, Line, Polygon,
    Radio, Checklist, Text,
    ClassificationAnnotation, ClassificationAnswer
)
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox import Client, LabelingFrontend, LabelImport, MALPredictionImport

### Replace with your API key
Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [3]:
API_KEY = None
client = Client(API_KEY)

## Supported annotations for tiled imagery

### Supported Python annotation types and NDJSON 

In [4]:
####### Point #######

# Python Annotation
point_annotation = ObjectAnnotation(
  name = "point_geo",
  value = Point(x=-99.21052827588443, y=19.405662413477728),
)

# NDJSON
point_annotation_ndjson = {
    "name": "point_geo",
    "point": {
         "x": -99.21052827588443,
         "y": 19.405662413477728
     }
}

In [5]:
####### Polyline #######
# Coordinates
coords = [[-99.2257610436396,19.418710777269197],[-99.22610441645385,19.341320919070036],[-99.1464419235419,19.341644802568833],[-99.14781541479901,19.420653145480912],[-99.22439165216781,19.42047504824125],[-99.1989820639114,19.414324129199997]]
line_points = []
line_points_ndjson = []

for sub in coords: 
  line_points.append(Point(x=sub[0], y=sub[1]))
  line_points_ndjson.append({"x":sub[0], "y":sub[1]})

# Python Annotation 
polyline_annotation = ObjectAnnotation(
  name = "polyline_geo",
  value = Line(points=line_points),
)


# NDJSON 
polyline_annotation_ndjson = {
    "name": "polyline_geo",
    "line": line_points_ndjson
}

In [6]:
####### Polygon #######
# Coordinates in the desired EPSG coordinate system
coords_polygon = [[-99.2103219219407,19.382085377579806],[-99.21375304694426,19.362328387802638],[-99.19006492831323,19.36297607074792],[-99.19281196088781,19.381437207301936],[-99.2103219219407,19.382085377579806]]
polygon_points = []
polygon_points_ndjson = []

for sub in coords_polygon: 
  polygon_points.append(Point(x=sub[0], y=sub[1]))
  polygon_points_ndjson.append({"x":sub[0], "y":sub[1]})

# Python Annotation 
polygon_annotation = ObjectAnnotation(
  name = "polygon_geo",
  value = Polygon(points=polygon_points),
)

# NDJSON 
polygon_annotation_ndjson = {
    "name": "polygon_geo",
    "polygon": polygon_points_ndjson
}

In [7]:
####### Bounding Box #######
coord_object = {
      "coordinates": [
        [
            [
                -99.210528,
                19.405662
            ],
            [
                -99.210528,
                19.400499
            ],
            [
                -99.205348,
                19.400499
            ],
            [
                -99.205348,
                19.405662
            ],
            [
                -99.210528,
                19.405662
            ]
          ]
        ]
      
    }


bbox_top_left = Point(x=-99.21052827588443, y=19.405662413477728)
bbox_bottom_right = Point(x=-99.20534818927473, y=19.400498983095076)

# Python Annotation
bbox_annotation = ObjectAnnotation(
  name = "bbox_geo",
  value = Rectangle(start=bbox_top_left, end=bbox_bottom_right)
)


# NDJSON
bbox_annotation_ndjson = {
    "name" : "bbox_geo",
    "bbox" : {
        'top': coord_object["coordinates"][0][1][1],
        'left': coord_object["coordinates"][0][1][0],
        'height': coord_object["coordinates"][0][3][1] - coord_object["coordinates"][0][1][1],        
        'width': coord_object["coordinates"][0][3][0] - coord_object["coordinates"][0][1][0]
    }
}


In [8]:
####### Classification - radio (single choice) #######

# Python Annotation 
radio_annotation = ClassificationAnnotation(
    name="radio_question_geo", 
    value=Radio(answer=ClassificationAnswer(name="first_radio_answer"))
)

# NDJSON 
radio_annotation_ndjson = {
    "name": "radio_question_geo",
    "answer": { "name": "first_radio_answer"}
}

In [9]:
####### Classification - Checklist (multi-choice) #######

coord_object_checklist = {
    "coordinates": [
        [
          [
              -99.173225,
              19.406409
          ],
          [
              -99.173225,
              19.398962
          ],
          [
              -99.192111,
              19.398962
          ],
          [
              -99.192111,
              19.406409
          ],
          [
              -99.173225,
              19.406409
          ]
      ]
    ]          
}

# Python Annotation
bbox_with_checklist_subclass = ObjectAnnotation(
    name="bbox_checklist_geo",
    value=Rectangle(
        start=Point(x=-99.17322500305542, y=19.40640857294981), # Top left
        end=Point(x=-99.19211050784055, y=19.398962049791287), # Bottom right
    ),
    classifications=[
        ClassificationAnnotation(
            name="checklist_class_name",
            value=Checklist(
                answer=[ClassificationAnswer(name="first_checklist_answer")]
            )
        )
    ]
)


# NDJSON 
bbox_with_checklist_subclass_ndjson = {
    "name": "bbox_checklist_geo", 
    "classifications": [{
        "name": "checklist_class_name",
        "answer": [
            { "name":"first_checklist_answer" }
        ]   
    }],
    "bbox": {
        'top': coord_object_checklist["coordinates"][0][1][1],
        'left': coord_object_checklist["coordinates"][0][1][0],
        'height': coord_object_checklist["coordinates"][0][3][1] - coord_object_checklist["coordinates"][0][1][1],        
        'width': coord_object_checklist["coordinates"][0][3][0] - coord_object_checklist["coordinates"][0][1][0]
    }
}

In [10]:
####### Classification free form text with bbox #######

coord_object_text ={
    "coordinates": [
        [
                [
                    -99.181809,
                    19.349418
                ],
                [
                    -99.181809,
                    19.357514
                ],
                [
                    -99.153996,
                    19.357514
                ],
                [
                    -99.153996,
                    19.349418
                ],
                [
                    -99.181809,
                    19.349418
                ]
            ]
        ]
}
# Python Annotation
bbox_with_free_text_subclass = ObjectAnnotation(
    name="bbox_text_geo",
    value=Rectangle(
        start=Point(x=-99.1818093234123, y=19.34941781372347), # Top left
        end=Point(x=-99.15399612545598, y=19.357514306583965), # Bottom right
    ),
    classifications=[
        ClassificationAnnotation(
            name="free_text_geo",
            value=Text(answer="sample text")
        )
    ]
)

# NDJSON 
bbox_with_free_text_subclass_ndjson = {
    "name":"bbox_text_geo",
    "classifications": [{
        "name": "free_text_geo",
        "answer": "sample text"
    }],
    "bbox": {
        'top': coord_object_text["coordinates"][0][1][1],
        'left': coord_object_text["coordinates"][0][1][0],
        'height': coord_object_text["coordinates"][0][3][1] - coord_object_text["coordinates"][0][1][1],        
        'width': coord_object_text["coordinates"][0][3][0] - coord_object_text["coordinates"][0][1][0]
    }
}

In [11]:
####### Classification - Checklist (multi-choice) #######

# Python Annotation
checklist_annotation = ClassificationAnnotation(
    name="checklist_question_geo",
    value=Checklist(answer = [
        ClassificationAnswer(name = "first_checklist_answer"),
        ClassificationAnswer(name = "second_checklist_answer"),
        ClassificationAnswer(name = "third_checklist_answer")
    ])
  )


# NDJSON
checklist_annotation_ndjson = {
  'name': 'checklist_question_geo',
  'answer': [
    {'name': 'first_checklist_answer'},
    {'name': 'second_checklist_answer'},
    {'name': 'third_checklist_answer'},
  ]
}

## Upload Annotations - putting it all together


### Step 1: Import data rows into Catalog

In [12]:

asset = [{
    "row_data": {
    "tile_layer_url": "https://storage.googleapis.com/labelbox-datasets/geospatial-sample-data/sample-geotiff.tif"
      },
    "global_key": str(uuid.uuid4()),
    "media_type": "TMS_GEO"
}]

dataset = client.create_dataset(name="geo_demo_dataset")
task = dataset.create_data_rows(asset)
task.wait_till_done()
print(task.errors)

None


### Step 2: Create/select an ontology
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations, and the tool names and classification instructions should match the `name`/`instructions` fields in your annotations to ensure the correct feature schemas are matched.

For example, when we create the bounding box annotation [above](https://colab.research.google.com/drive/1_sPVTktS1AVvFMCg-GR4q_g2ukQaR4Oq#scrollTo=2IZXIWP2UcOJ&line=2&uniqifier=1), we provided the `name` as `bbox_checklist_geo`. Now, when we setup our ontology, we must ensure that the name of my bounding box tool is also `bbox_checklist_geo`. The same alignment must hold true for the other tools and classifications we create in our ontology.

In [13]:
ontology_builder = OntologyBuilder(
    tools=[
        Tool(tool=Tool.Type.POINT, name="point_geo"),
        Tool(tool=Tool.Type.LINE, name="polyline_geo"),
        Tool(tool=Tool.Type.POLYGON, name="polygon_geo"),
        Tool(tool=Tool.Type.BBOX, name="bbox_geo"), 
        Tool( 
          tool=Tool.Type.BBOX, 
          name="bbox_checklist_geo",
          classifications=[
                Classification(
                    class_type=Classification.Type.CHECKLIST,
                    instructions="checklist_class_name",
                    options=[
                      Option(value="first_checklist_answer")
                    ]
                ),
            ]
          ),
        Tool( 
          tool=Tool.Type.BBOX, 
          name="bbox_text_geo",
          classifications=[
                Classification(
                    class_type=Classification.Type.TEXT,
                    instructions="free_text_geo"
                ),
            ]
          )    
      ],
      classifications = [
          Classification(
              class_type=Classification.Type.CHECKLIST, 
              instructions="checklist_question_geo",
              options=[
                  Option(value="first_checklist_answer"),
                  Option(value="second_checklist_answer"), 
                  Option(value="third_checklist_answer")
              ]
          ), 
          Classification(
              class_type=Classification.Type.RADIO, 
              instructions="radio_question_geo",
              options=[
                  Option(value="first_radio_answer")
              ]
          )
          
      ]
)

ontology = client.create_ontology("Ontology Geospatial Annotations", ontology_builder.asdict())

### Step 3: Create a labeling project
Connect the ontology to the labeling project 

In [22]:
# Project defaults to batch mode with benchmark quality settings if this argument is not provided
# Queue mode will be deprecated once dataset mode is deprecated

project = client.create_project(name="geo_project_demo",
                                    queue_mode=QueueMode.Batch,
                                    media_type=MediaType.Geospatial_Tile)


project.setup_editor(ontology)

######################### DATASET CONSENSUS OPTION ########################
#Note that dataset base projects will be deprecated in the near future.

#To use Datasets/Consensus instead of Batches/Benchmarks use the following query: 
#In this case, 10% of all data rows need to be annotated by three labelers.

# dataset_project = client.create_project(name="dataset-test-project",
#                                 description="a description",
#                                 media_type=MediaType.Text,
#                                 auto_audit_percentage=0.1,
#                                 auto_audit_number_of_labels=3,
#                                 queue_mode=QueueMode.Dataset)

# dataset_project.datasets.connect(dataset)

### Step 4: Send a batch of data rows to the project 

In [23]:
# Setup Batches and Ontology

# Create a batch to send to your MAL project
batch = project.create_batch(
  "first-batch-geo-demo", # Each batch in a project must have a unique name
  dataset.export_data_rows(), # Paginated collection of data row objects
  5 # priority between 1(Highest) - 5(lowest)
)

print("Batch: ", batch)

Batch:  <Batch {
    "consensus_settings_json": "{\"numberOfLabels\":1,\"coveragePercentage\":0}",
    "created_at": "2023-01-03 23:20:49+00:00",
    "name": "first-batch-geo-demo",
    "size": 1,
    "uid": "421bee80-8bbd-11ed-a6f6-1fd0e0fcd05c",
    "updated_at": "2023-01-03 23:20:49+00:00"
}>


### Step 5: Create the annotations payload 
Create the annotations payload using the snippets of code above

Labelbox support two formats for the annotations payload: NDJSON and Python Annotation types. Both are described below. 


#### Python annotations
Here we create the complete label ndjson payload of annotations only using python annotation format. There is one annotation for each reference to an annotation that we created [above](https://colab.research.google.com/drive/1_sPVTktS1AVvFMCg-GR4q_g2ukQaR4Oq#scrollTo=AmKgfZ8ZALT_).

In [16]:
label = Label(
    data=ImageData(
        uid= next(dataset.export_data_rows()).uid
    ),
    annotations = [
        point_annotation,
        polyline_annotation,
        polygon_annotation,
        bbox_annotation,
        radio_annotation,
        bbox_with_checklist_subclass,  
        bbox_with_free_text_subclass,
        checklist_annotation
    ]
)

# Convert our label from a Labelbox class object to the underlying NDJSON format required for upload 
label_ndjson = list(NDJsonConverter.serialize([label]))
label_ndjson

[{'uuid': '9dccd23f-7485-4a09-b7ba-b4f6f6e6da31',
  'dataRow': {'id': 'clcgupd6i0c5r077fagkp9vx6'},
  'name': 'point_geo',
  'classifications': [],
  'point': {'x': -99.21052827588443, 'y': 19.405662413477728}},
 {'uuid': '20a8bcfa-5114-461e-981f-2f12f9e5e5d5',
  'dataRow': {'id': 'clcgupd6i0c5r077fagkp9vx6'},
  'name': 'polyline_geo',
  'classifications': [],
  'line': [{'x': -99.2257610436396, 'y': 19.418710777269197},
   {'x': -99.22610441645385, 'y': 19.341320919070036},
   {'x': -99.1464419235419, 'y': 19.341644802568833},
   {'x': -99.14781541479901, 'y': 19.420653145480912},
   {'x': -99.22439165216781, 'y': 19.42047504824125},
   {'x': -99.1989820639114, 'y': 19.414324129199997}]},
 {'uuid': '65b0bde8-d16a-414d-8e95-7f5f193fc986',
  'dataRow': {'id': 'clcgupd6i0c5r077fagkp9vx6'},
  'name': 'polygon_geo',
  'classifications': [],
  'polygon': [{'x': -99.2103219219407, 'y': 19.382085377579806},
   {'x': -99.21375304694426, 'y': 19.362328387802638},
   {'x': -99.19006492831323, 'y

### NDJSON annotations
Here we create the complete label NDJSON payload of annotations only using NDJSON format. There is one annotation for each reference to an annotation that we created [above](https://colab.research.google.com/drive/1_sPVTktS1AVvFMCg-GR4q_g2ukQaR4Oq#scrollTo=AmKgfZ8ZALT_).

In [17]:
label_ndjson_method2 = []

for annotations in [point_annotation_ndjson,
                    polyline_annotation_ndjson,
                    polygon_annotation_ndjson,
                    bbox_annotation_ndjson,
                    radio_annotation_ndjson,
                    bbox_with_checklist_subclass_ndjson,  
                    bbox_with_free_text_subclass_ndjson,
                    checklist_annotation_ndjson]:
  annotations.update({
      'uuid' : str(uuid.uuid4()),
      'dataRow': {
          'id':  next(dataset.export_data_rows()).uid
      }
  })
  label_ndjson_method2.append(annotations)
  

In [18]:
label_ndjson_method2

[{'name': 'point_geo',
  'point': {'x': -99.21052827588443, 'y': 19.405662413477728},
  'uuid': '9f3225f9-63bd-4c88-941d-d75ece46945a',
  'dataRow': {'id': 'clcgupd6i0c5r077fagkp9vx6'}},
 {'name': 'polyline_geo',
  'line': [{'x': -99.2257610436396, 'y': 19.418710777269197},
   {'x': -99.22610441645385, 'y': 19.341320919070036},
   {'x': -99.1464419235419, 'y': 19.341644802568833},
   {'x': -99.14781541479901, 'y': 19.420653145480912},
   {'x': -99.22439165216781, 'y': 19.42047504824125},
   {'x': -99.1989820639114, 'y': 19.414324129199997}],
  'uuid': '22e296af-950d-41eb-b7c2-8f718e480397',
  'dataRow': {'id': 'clcgupd6i0c5r077fagkp9vx6'}},
 {'name': 'polygon_geo',
  'polygon': [{'x': -99.2103219219407, 'y': 19.382085377579806},
   {'x': -99.21375304694426, 'y': 19.362328387802638},
   {'x': -99.19006492831323, 'y': 19.36297607074792},
   {'x': -99.19281196088781, 'y': 19.381437207301936},
   {'x': -99.2103219219407, 'y': 19.382085377579806}],
  'uuid': 'bd2b0960-d320-4807-8954-dec1afe

### Step 6: Upload annotations to a project as pre-labels or complete labels


#### Model-Assisted Labeling (MAL)
For the purpose of this tutorial only run one of the label_ndjosn  annotation type tools at the time (NDJSON or Annotation types). Delete the previous labels before uploading labels that use the 2nd method (ndjson)

In [24]:
# Upload MAL label for this data row in project
upload_job = MALPredictionImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="mal_import_job"+str(uuid.uuid4()), 
    ### use label_ndjson_method2 if labels were created using NDJSON tools
    predictions=label_ndjson)

upload_job.wait_until_done();
print("Errors:", upload_job.errors)
print("   ")

Errors: []
   


#### Label Import

In [20]:
# Upload label for this data row in project 
upload_job = LabelImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="label_geo_import_job"+str(uuid.uuid4()),  
    # user label_ndjson if labels were created using python annotation tools
    labels=label_ndjson_method2)

upload_job.wait_until_done();
print("Errors:", upload_job.errors)

Errors: []


### Optional deletions for cleanup

In [21]:
#upload_job
# project.delete()
# dataset.delete()